### NOTEBOOK: 02_Incremental_Optimization.ipynb

    Data: 19/11/2025
    Autor: Debora Rebula Klein
    Versão: 1.0
    Status: PROPOSAL

---

### 1. BUSINESS UNDERSTANDING 

#### 1.1 Contexto de Negócio

| Problema | Impacto Financeiro | Oportunidade | Objetivo de Negócio |
| :--- | :--- | :--- | :--- |
| Custo operacional do pipeline completo (full refresh) é insustentável em escala | ~$12.00/mês para processar dados majoritariamente históricos | 95% dos dados são estáticos - apenas 5% são novos diariamente | Reduzir custo operacional em 80-90% mantendo mesma qualidade |


#### 1.2 Justificativa Econômica

| Métrica | Full Refresh | Incremental | Economia |
| :--- | :--- | :--- | :--- |
| Custo Mensal | $12.00 | $0.75 - $2.40 | 80-94% |
| Tempo Processamento | 15-20min | 3-5min | 70-80% |
| Consumo Compute | 16 cores | 2-4 cores | 75-87% |

#### 1.3 Critérios de Sucesso

- Redução de custo ≥80%

- Manter 100% da qualidade dos dados

- Zero impacto nos dashboards existentes

- Tempo de processamento <5min

### 2. DATA UNDERSTANDING

#### 2.1 Análise do Volume de Dados

In [0]:
%sql
-- PERFIL DOS DADOS ATUAIS
WITH data_profile AS (
  SELECT 
    'orders' as tabela,
    COUNT(*) as total_registros,
    COUNT(DISTINCT DATE(created_at)) as dias_cobertura,
    MIN(created_at) as data_minima,
    MAX(created_at) as data_maxima
  FROM tbl_fact_orders_bronze
)
SELECT 
  tabela,
  total_registros,
  dias_cobertura,
  -- Estimativa de crescimento diário
  total_registros / dias_cobertura as media_diaria
FROM data_profile;

#### 2.2 Padrão de Atualização

| Tabela | Frequência de Atualização | Mudanças Históricas | Viabilidade Incremental |
| :--- | :--- | :--- | :---|
| tbl_fact_orders_bronze | Apenas inserts | 0% - dados imutáveis após criação | ALTA |
| tbl_fact_deliveries_bronze | Updates raros (status) |	<1% - apenas status pendente→entregue | ALTA |
| tbl_dim_stores_bronze | Updates esporádicos | ~2% - mudança de segmento/hub | MÉDIA (precisa MERGE) |


### 3. DATA PREPARATION

#### 3.1 Estratégia de Incrementalidade por Camada

##### Bronze Layer - Approach

Vamos adicionar colunas de controle para incrementalidade.

In [0]:
%sql
-- IMPLEMENTAÇÃO: CONTROLE DE INCREMENTALIDADE
ALTER TABLE tbl_fact_orders_bronze 
ADD COLUMNS (
    _loaded_date DATE COMMENT 'Data do carregamento incremental',
    _batch_id STRING COMMENT 'Identificador do lote'
);

-- QUERY INCREMENTAL BASE
CREATE OR REPLACE TEMPORARY VIEW novos_pedidos AS
SELECT 
    order_id,
    order_moment_created as created_at,
    order_amount as subtotal, 
    order_delivery_fee as delivery_fee,
    store_id,
    CURRENT_DATE() as _loaded_date,
    UUID() as _batch_id
FROM '/Volumes/.../orders.csv'
WHERE TO_DATE(order_moment_created) = CURRENT_DATE() - INTERVAL 1 DAY;

##### 3.2 Silver Layer - Strategy

Para a camada Silver, usaremos uma abordagem de UNION ALL para preservar o histórico e processar apenas os novos dados.

In [0]:
%sql
-- EXEMPLO DE ABORDAGEM INCREMENTAL NA SILVER
CREATE OR REPLACE TEMPORARY VIEW silver_incremental AS
-- Manter os dados existentes, exceto os que serão atualizados
SELECT * FROM tbl_fact_pedidos_silver
WHERE order_id NOT IN (SELECT order_id FROM novos_pedidos)

UNION ALL

-- Adicionar os novos pedidos processados
SELECT 
    order_id,
    store_id,
    driver_id,
    payment_method,
    subtotal_bruto,
    delivery_fee_cliente,
    created_at_ts_str,
    gmv_total,
    receita_comissao,
    receita_liquida_plataforma,
    cogs_logistico_simulado,
    cogs_transacao_simulado,
    lucro_bruto_unitario
FROM processamento_novos_pedidos;

### 4. MODELING 

#### 4.1 Arquitetura Proposta

In [0]:
BEFORE: CSV → [BRONZE Full] → [SILVER Full] → [GOLD Full]
AFTER:  CSV → [BRONZE Incremental] → [SILVER Incremental] → [GOLD Incremental]
                    ↑                      ↑                     ↑
              Controle de lote        Processamento        Atualização
              _loaded_date           apenas novos          dimensional

#### 4.2 Modelo de Controle de Processamento

In [0]:
%sql
-- TABELA DE CONTROLE DE INCREMENTALIDADE
CREATE TABLE IF NOT EXISTS ctrl_incremental_processing (
    table_name STRING,
    last_processed_date DATE,
    last_batch_id STRING,
    records_processed INT,
    status STRING,
    processed_at TIMESTAMP
);

-- QUERY PARA DETERMINAR PRÓXIMA EXECUÇÃO
SELECT 
    table_name,
    last_processed_date,
    COALESCE(last_processed_date, '2020-01-01') as start_date,
    CURRENT_DATE() - INTERVAL 1 DAY as end_date
FROM ctrl_incremental_processing
WHERE table_name = 'tbl_fact_orders_bronze';33

### 5. EVALUATION 

#### 5.1 Plano de Validação

Completude dos Dados

In [0]:
%sql
-- COUNT comparativo full vs incremental
SELECT 
    (SELECT COUNT(*) FROM full_table) as count_full,
    (SELECT COUNT(*) FROM incremental_table) as count_inc,
    ABS(count_full - count_inc) as difference

Consistência Financeira

In [0]:
%sql
-- Soma de GMV e margens
SELECT 
    full.gmv_total as gmv_full,
    inc.gmv_total as gmv_inc,
    ABS(full.gmv_total - inc.gmv_total) / full.gmv_total as pct_diff
FROM full_aggregate full, incremental_aggregate inc

#### 5.2 Critérios de Go/No-Go

In [0]:
%sql
-- VALIDAÇÃO AUTOMATIZADA
WITH validation_results AS (
    SELECT 
        'completude' as teste,
        CASE WHEN count_diff < 100 THEN 'PASS' ELSE 'FAIL' END as status
    FROM (
        SELECT ABS(full_count - inc_count) as count_diff
        FROM table_counts_comparison
    )
    UNION ALL
    SELECT 
        'consistencia_financeira' as teste,
        CASE WHEN gmv_pct_diff < 0.001 THEN 'PASS' ELSE 'FAIL' END as status  
    FROM gmv_comparison
)
SELECT 
    COUNT(*) as total_testes,
    SUM(CASE WHEN status = 'PASS' THEN 1 ELSE 0 END) as testes_aprovados,
    CASE 
        WHEN total_testes = testes_aprovados THEN '✅ GO'
        ELSE '❌ NO-GO - Revisar implementação'
    END as decisao
FROM validation_results;

### 6. DEPLOYMENT

#### 6.1 Plano de Transição

| Fase | Período | Ação | Validação | Rollback |
| :--- | :--- | :--- | :--- | :--- |
| 1 | Semana 1 | Implementar Bronze incremental em paralelo | Comparar counts diariamente | Manter pipeline full como primary |
| 2 | Semana 2 | Implementar Silver incremental | Dashboards em paralelo - métricas financeiras | Switch back para Silver full se divergência > 0.1% |
| 3 | Semana 3 | Cutover completo para incremental | Monitoramento intensivo 7 dias | Procedimento documentado para restauração full |


#### 6.2 Monitoramento Pós-Implantação

In [0]:
%sql
-- DASHBOARD DE HEALTH CHECK
CREATE OR REPLACE VIEW monitor_incremental_health AS
SELECT 
    date,
    table_name,
    execution_time_seconds,
    records_processed,
    cost_dbu,
    data_freshness_hours,
    CASE 
        WHEN execution_time_seconds > 300 THEN 'ALERTA'
        WHEN data_freshness_hours > 26 THEN 'ALERTA' 
        ELSE 'NORMAL'
    END as health_status
FROM incremental_process_logs
WHERE date >= CURRENT_DATE() - 7;

### 7. BENEFÍCIOS ESPERADOS 

#### 7.1 Impacto Financeiro Direto

| Métrica | Antes | Depois | Economia |
| :--- | :--- | :--- | :--- |
| Custo Mensal | $12.00 | $0.75 - $2.40 | $9.60 - $11.25 |
| Custo Anual | $144.00 | $9.00 - $28.80 | $115.20 - $135.00 |
| ROI (1 ano) | - | 500-1500% | - |

#### 7.2 Benefícios Operacionais

- Escalabilidade: Suporte a 10x mais dados sem aumento proporcional de custo

- Performance: Dashboards atualizados mais rapidamente

- Confiabilidade: Processos mais curtos = menor janela de falha

- Manutenibilidade: Debug mais fácil com processamento por lotes

- Sustentabilidade: Menor consumo computacional = menor impacto ambiental

#### 7.3 Benefícios Estratégicos

- Capacidade de expansão: Economia permite incluir novas fontes de dados

- Cultura data-driven: Custo baixo permite experimentação e POCs

- Resiliência financeira: Pipeline sustentável mesmo em cenários de restrição orçamentária

### 8. PRÓXIMOS PASSOS 

#### 8.1 Imediatos (Sprint 1)

Aprovação do design com stakeholders

Desenvolvimento Bronze incremental

Plano de teste de validação

#### 8.2 Curto Prazo (Sprint 2)

Implementação Silver incremental

Validação paralela com produção

Documentação de operação

#### 8.3 Médio Prazo (Sprint 3)

Cutover controlado para incremental

Monitoramento contínuo

Retrospectiva e lições aprendidas

##### STATUS: READY FOR DEVELOPMENT
##### APPROVALS: [ ] Business [ ] Tech Lead [ ] Data Engineering